In [1]:
import tensorflow as tf
import numpy as np
import nengo_dl
import nengo

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [3]:
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


# TF Model definition

In [4]:
# input
inp = tf.keras.Input(shape=(28, 28, 1))

# convolutional layers
conv0 = tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=3,
    activation=tf.nn.relu,
)(inp)

# Default pool_size = (2,2), padding = "valid", data_format = "channels_last".
max_pool0 = tf.keras.layers.MaxPool2D()(conv0) 

conv1 = tf.keras.layers.Conv2D(
    filters=64,
    kernel_size=3,
    strides=2,
    activation=tf.nn.relu,
)(max_pool0)

max_pool1 = tf.keras.layers.MaxPool2D()(conv1) 

conv2 = tf.keras.layers.Conv2D(
    filters=64,
    kernel_size=3,
    strides=2,
    activation=tf.nn.relu,
)(max_pool1)


# fully connected layer
flatten = tf.keras.layers.Flatten()(conv2)
dense = tf.keras.layers.Dense(units=10, activation="softmax")(flatten)

model = tf.keras.Model(inputs=inp, outputs=dense)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 6, 64)          18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 64)                0     

# TF Model Compilation and Fitting

In [6]:
model.compile(
  optimizer=tf.optimizers.Adam(0.001),
  loss=tf.losses.SparseCategoricalCrossentropy(),
  metrics=[tf.metrics.sparse_categorical_accuracy])
model.fit(train_images, train_labels, epochs=4)

Epoch 1/4
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3602 - sparse_categorical_accuracy: 0.9233
Epoch 2/4
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0812 - sparse_categorical_accuracy: 0.9749
Epoch 3/4
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0657 - sparse_categorical_accuracy: 0.9793
Epoch 4/4
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0553 - sparse_categorical_accuracy: 0.9829


# TF Model evalution

In [7]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 2s 5ms/step - loss: 0.0800 - sparse_categorical_accuracy: 0.9739


[0.07999689877033234, 0.9739000201225281]

# Conversion from TF to spiking Nengo DL model (max_to_avg_pool=False)

In [8]:
n_steps = 40
np.random.seed(100)
ndl_model_1 = nengo_dl.Converter(model, 
                               swap_activations={tf.nn.relu: nengo.SpikingRectifiedLinear()},
                               scale_firing_rates=5,
                               max_to_avg_pool=False,
                               synapse=0.005)

with ndl_model_1.net:
  nengo_dl.configure_settings(stateful=False)

/home/rgaurav/miniconda3/envs/latest-nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:326: UserWarning: Cannot convert max pooling layers to native Nengo objects; consider setting max_to_avg_pool=True to use average pooling instead. Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")
/home/rgaurav/miniconda3/envs/latest-nengo-tf/lib/python3.7/site-packages/nengo_dl/converter.py:588: UserWarning: Activation type <function softmax at 0x2aaeb5775cb0> does not have a native Nengo equivalent; falling back to a TensorNode
  "falling back to a TensorNode" % activation


## Nengo-DL model test data creation and inference

In [9]:
ndl_test_images = np.tile(
  test_images.reshape((test_images.shape[0], 1, -1)), (1, n_steps, 1))
ndl_input_1 = ndl_model_1.inputs[inp]
ndl_output_1 = ndl_model_1.outputs[dense]

In [10]:
with nengo_dl.Simulator(
  ndl_model_1.net, minibatch_size=100) as sim:
  data1 = sim.predict({ndl_input_1: ndl_test_images})

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               


## Nengo-DL model accuracy

In [11]:
acc = 0
for pred, true in zip(data1[ndl_output_1][:, -1, :], test_labels):
  if np.argmax(pred) == true:
    acc += 1
print(acc/10000)

0.6669


# Conversion from TF to spiking Nengo DL model (max_to_avg_pool=True)

In [12]:
n_steps = 40
np.random.seed(100)
ndl_model_2 = nengo_dl.Converter(model, 
                               swap_activations={tf.nn.relu: nengo.SpikingRectifiedLinear()},
                               scale_firing_rates=5,
                               max_to_avg_pool=True,
                               synapse=0.005)

with ndl_model_2.net:
  nengo_dl.configure_settings(stateful=False)

## Nengo-DL model test data creation and inference

In [13]:
ndl_test_images = np.tile(
  test_images.reshape((test_images.shape[0], 1, -1)), (1, n_steps, 1))
ndl_input_2 = ndl_model_2.inputs[inp]
ndl_output_2 = ndl_model_2.outputs[dense]

In [14]:
with nengo_dl.Simulator(
  ndl_model_2.net, minibatch_size=100) as sim:
  data2 = sim.predict({ndl_input_2: ndl_test_images[:200]})

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               


## Nengo-DL model accuracy

In [15]:
acc = 0
for pred, true in zip(data2[ndl_output_2][:, -1, :], test_labels):
  if np.argmax(pred) == true:
    acc += 1
print(acc/200)

0.45
